# Deploy Llama3 with TensorRT-LLM🤙

Welcome!

In this notebook, we will walk through on converting Mistral into the TensorRT format. TensorRT-LLM provides users with an easy-to-use Python API to define Large Language Models (LLMs) and build TensorRT engines that contain state-of-the-art optimizations to perform inference efficiently on NVIDIA GPUs.

Once the TensorRT engine is build, you can use the run.py script provided at the end of this notebook or use this engine as in input to the Triton Inference Server. 

See the [Github repo](https://github.com/NVIDIA/TensorRT-LLM) for more examples and documentation!

Deployment powered by [Brev.dev](https://x.com/brevdev) 

### Step 1 - Install TensorRT-LLM

We first install TensorRT-LLM. 

In [1]:
!pip install tensorrt_llm -U --pre --extra-index-url https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 871.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.0 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.6/975.6 kB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.7/704.7 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 9.0 MB/s eta 0:0

### Step 2 - Download Llama3 model weights

Llama3 is a gated model which means you'll need to request approval on their respository and generate a HF token. This usually takes about 20 minutes!

In [2]:
import huggingface_hub

In [3]:
huggingface_hub.login("<ENTER TOKEN HERE>")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
huggingface_hub.snapshot_download("meta-llama/Meta-Llama-3-8B-Instruct", local_dir="llama3-hf")

Fetching 17 files: 100%|██████████| 17/17 [01:34<00:00,  5.58s/it]


'/root/verb-workspace/llama3-hf'

### Step 3 - Convert checkpoints into safetensors and build the TRT engine

There are 2 substeps here. The first is converting the raw huggingface model into safetensors which is a safe and fast format for storing tensors. 

Next we build the TensorRT engine. This is where the magic happens. We take the converted safetensors model and convert it into a `TensorRT engine`. Engines are optimized versions of models built to run lightening fast on the current machine.  

In [6]:
!wget -L https://raw.githubusercontent.com/NVIDIA/TensorRT-LLM/main/examples/llama/convert_checkpoint.py

--2024-04-23 14:34:07--  https://raw.githubusercontent.com/NVIDIA/TensorRT-LLM/main/examples/llama/convert_checkpoint.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17571 (17K) [text/plain]
Saving to: ‘convert_checkpoint.py’

convert_checkpoint. 100%[===================>]  17.16K  --.-KB/s    in 0.001s  

2024-04-23 14:34:07 (23.8 MB/s) - ‘convert_checkpoint.py’ saved [17571/17571]



In [8]:
!python convert_checkpoint.py --model_dir llama3-hf \
    --output_dir ./llama3-safetensors \
    --dtype bfloat16

[TensorRT-LLM] TensorRT-LLM version: 0.10.0.dev2024041600
0.10.0.dev2024041600
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:05<00:00,  1.28s/it]
Weights loaded. Total time: 00:00:17
Total time of converting checkpoints: 00:01:11


In [11]:
!trtllm-build --checkpoint_dir llama3-safetensors \
    --output_dir ./llama3engine_bf16_1gpu \
    --gpt_attention_plugin bfloat16 \
    --gemm_plugin bfloat16

[TensorRT-LLM] TensorRT-LLM version: 0.10.0.dev2024041600
[04/23/2024-14:38:18] [TRT-LLM] [I] Set bert_attention_plugin to float16.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set gpt_attention_plugin to bfloat16.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set gemm_plugin to bfloat16.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set nccl_plugin to float16.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set lookup_plugin to None.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set lora_plugin to None.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set moe_plugin to float16.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set mamba_conv1d_plugin to float16.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set context_fmha to True.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set context_fmha_fp32_acc to False.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set paged_kv_cache to True.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set remove_input_padding to True.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set use_custom_all_reduce to True.
[04/23/2024-14:38:18] [TRT-LLM] [I] Set multi_block_mode to False.


### Step 4 - Run the model using the example script!

In [13]:
!git clone https://github.com/NVIDIA/TensorRT-LLM.git

Cloning into 'TensorRT-LLM'...
remote: Enumerating objects: 13921, done.
remote: Counting objects: 100% (6026/6026), done.
remote: Compressing objects: 100% (1392/1392), done.
remote: Total 13921 (delta 4815), reused 5280 (delta 4600), pack-reused 7895
Receiving objects: 100% (13921/13921), 203.58 MiB | 41.30 MiB/s, done.
Resolving deltas: 100% (9982/9982), done.
Updating files: 100% (2120/2120), done.


In [20]:
!python ./TensorRT-LLM/examples/run.py --engine_dir=llama3engine_bf16_1gpu \
    --max_output_len 100 \
    --tokenizer_dir llama3-hf \
    --input_text "How do I count to nine in French?"

[TensorRT-LLM] TensorRT-LLM version: 0.10.0.dev2024041600
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[TensorRT-LLM][INFO] Engine version 0.10.0.dev2024041600 found in the config file, assuming engine(s) built by new builder API.
[TensorRT-LLM][WARNING] [json.exception.type_error.302] type must be string, but is null
[TensorRT-LLM][WARNING] Optional value for parameter quant_algo will not be set.
[TensorRT-LLM][WARNING] [json.exception.type_error.302] type must be string, but is null
[TensorRT-LLM][WARNING] Optional value for parameter kv_cache_quant_algo will not be set.
[TensorRT-LLM][WARNING] [json.exception.out_of_range.403] key 'num_medusa_heads' not found
[TensorRT-LLM][WARNING] Optional value for parameter num_medusa_heads will not be set.
[TensorRT-LLM][WARNING] [json.exception.out_of_range.403] key 'max_draft_len' not found
[TensorRT-LLM][WARNING] Optional value for parameter max_draft_len will not be se